# Digitale Medien

In [1]:
# loading libs
import pandas as pd
import numpy as np
import pprint as pp
import datetime
import Stemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

In [5]:
# politiker = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/politikR_demojized_reinigung.csv",
#                         index_col=0)

# öffentlicher = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/örr_demojized.csv",
#                            index_col=0)

# überregionale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/überregio_demojized.csv",
#                             index_col=0)
# überregionale = überregionale.drop(columns='...1')

digitale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/digitale_demojized.csv",
                       index_col=0)
digitale = digitale.drop(columns='...1')

In [6]:
digitale

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,hashtagList,cashtags,user,userID,userLoc,followerAmount,erreichbarkeit,normenWerte,geschäftsmodell,text
0,2022-10-27T06:26:03Z,1.585518e+18,@CDU Seit Jahresbeginn haben Parteien insg. 10...,3,13,65,3,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,@cdu seit jahresbeginn haben parteien insg 10 ...
1,2022-10-26T16:57:38Z,1.585315e+18,Die umstrittene Klimastiftung MV möchte verhin...,4,55,129,5,NaN,NaN,NaN,...,['NordStream2'],NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,die umstrittene klimastiftung mv möchte verhin...
2,2022-10-26T16:20:51Z,1.585305e+18,"""Bitte gehen Sie weiter, es gibt hier nichts z...",1,19,56,0,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,bitte gehen sie weiter es gibt hier nichts zu ...
3,2022-10-26T14:52:11Z,1.585283e+18,+++ Großspenden-Ticker +++\n\n55.000 Euro an d...,5,50,280,1,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,großspenden-ticker \n\n55000 euro an die @cdu...
4,2022-10-26T12:44:57Z,1.585251e+18,Hier haben wir 2021 die Hintergründe beschrieb...,2,25,136,1,NaN,NaN,NaN,...,NaN,NaN,a_watch,35142791.0,Hamburg,107748,3.0,2.0,3.0,hier haben wir 2021 die hintergründe beschrieb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89155,2022-01-01T10:26:52Z,1.477225e+18,"""Dachte, du würdest ewig leben"": So trauern di...",0,3,6,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,dachte du würdest ewig leben so trauern die st...
89156,2022-01-01T09:51:06Z,1.477216e+18,Deutscher Student bei Messerstecherei in Tel A...,0,1,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,deutscher student bei messerstecherei in tel a...
89157,2022-01-01T08:14:34Z,1.477192e+18,Drosten vorsichtig optimistisch für weitere Co...,0,0,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,drosten vorsichtig optimistisch für weitere co...
89158,2022-01-01T07:28:32Z,1.477180e+18,Polizei zieht nach Silvesternacht Bilanz - Töd...,0,0,1,0,NaN,NaN,NaN,...,NaN,NaN,WEBDENews,68724671.0,München,4695,1.0,1.0,2.0,polizei zieht nach silvesternacht bilanz tödl...


In [2]:
categoriesDigital = {
    'proteste' : ['proteste', 'widerstand', 'corona-maßnahmen', 'protest', 'corona-politik', 'afghanistan'],
    
    'unglück' : ['unfall', 'verletzt', 'einsatz', 'tote', 'feuerwehr', 'verletzte', 'stirbt', 'polizisten',
                 'a45-sperrung', 'feuer', 'brand'],
    
    'covid' : ['corona', 'lauterbach', 'impfpflicht', 'pandemie', 'impfung', 'maskenpflicht',
               'gesundheitsminister', 'omikron', 'maske', 'impfen', 'rki', 'nebenwirkungen', 'ungeimpfte',
               'masken', 'geimpft', 'coronavirus', 'drosten'],
    
    'öpnvVerkehr' : ['bahn', 'fahren', '9-euro-ticket', 'urlaub', 'chaos'],
    
    'wetterKlima' : ['sommer', 'wasser', 'klimawandel', 'wetter', 'hitze', 'grad', 'temperaturen'],
    
    'europa' : ['polen', 'europa', 'österreich', 'europäische', 'spanien', 'ungarn', 'brüssel'],
    
    'kinderbetreuung' : ['kinder', 'junge', 'eltern', 'kindern', 'jungen', 'schulen', 'jugendliche',
                         'schule', 'mädchen', 'schüler'],
    
    'energiekrise' : ['gas', 'habeck', 'bundesregierung', 'energie', 'energiekrise', 'energiewende', 'öl',
                      'wirtschaftsminister', 'atomkraft', 'erdgas', 'strom', 'gasumlage', 'sparen',
                      'gaskrise', 'atomkraftwerke', 'inflation', 'preise', 'steigen', 'steigt', 'teurer',
                      'kosten', 'energiepreise', 'preis', 'kunden', 'steigende', 'lebensmittel'],
    
    'ukraine' : ['ukraine', 'russische', 'russischen', 'ukraine-krieg', 'russland', 'ukrainische', 'kiew',
                 'angriff', 'selenskyj', 'ukrainischen', 'soldaten', 'russischer', 'truppen', 'armee', 'ukrainer',
                 'russlands', 'russen', 'mariupol', 'ukrainekrieg', 'ukraine', 'russland', 'putin', 'krieg',
                 'putins', 'sanktionen', 'nato', 'waffen', 'frieden', 'moskau', 'ukrainekrieg', 'russlands',
                 'kreml', 'waffenlieferungen', 'russen', 'krieges'],
    
    'fakenews' : ['querdenker', 'falsch', 'lügen', 'fehler', 'wahrheit', 'fake', 'hass', 'news',
                  'propaganda', 'meinung', 'fakten', 'gegenteil', 'desinformation'],
    
    'finanzielles' : ['euro', 'millionen', 'geld', 'milliarden', '€', 'wert', 'spenden', 'dollar', 'kosten', 'mio']
}